In [1]:
import requests

/Users/benlevine/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
from bs4 import BeautifulSoup

In [5]:
soup = BeautifulSoup(data.text)

In [6]:
standings_table = soup.select('table.stats_table') [0]

In [7]:
links =standings_table.find_all('a')

In [8]:
links = [l.get("href") for l in links]

In [9]:
links = [l for l in links if '/squad' in l]

In [10]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [11]:
team_urls

['https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats',
 'https://fbref.com/en/squads/d3fd31cc/Everton-Stats',
 'https://fbref.com/en/squads/

In [12]:
team_url = team_urls[0]

In [13]:
data = requests.get(team_url)

In [14]:
import pandas as pd

matches =pd.read_html(data.text, match="Scores & Fixtures")


/var/folders/st/4hj4v2pn7hj60r64jss1y9nw0000gn/T/ipykernel_3308/1536848166.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches =pd.read_html(data.text, match="Scores & Fixtures")


In [15]:
#matches[0]

In [16]:
soup = BeautifulSoup(data.text)

In [17]:
links = soup.find_all('a')

In [18]:
links = [l.get("href") for l in links]

In [19]:
links = [l for l in links if l and 'all_comps/shooting' in l ]

In [20]:
links

['/en/squads/822bd0ba/2024-2025/matchlogs/all_comps/shooting/Liverpool-Match-Logs-All-Competitions',
 '/en/squads/822bd0ba/2024-2025/matchlogs/all_comps/shooting/Liverpool-Match-Logs-All-Competitions',
 '/en/squads/822bd0ba/2024-2025/matchlogs/all_comps/shooting/Liverpool-Match-Logs-All-Competitions',
 '/en/squads/822bd0ba/2024-2025/matchlogs/all_comps/shooting/Liverpool-Match-Logs-All-Competitions']

In [21]:
data = requests.get(f"https://fbref.com{links[0]}")

In [22]:
shooting = pd.read_html(data.text, match="Shooting")[0]

/var/folders/st/4hj4v2pn7hj60r64jss1y9nw0000gn/T/ipykernel_3308/2517714723.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


In [23]:
shooting.head() 

For Liverpool                                                              \
           Date   Time            Comp         Round  Day Venue Result   GF   
0    2024-08-17  12:30  Premier League   Matchweek 1  Sat  Away      W  2.0   
1    2024-08-25  16:30  Premier League   Matchweek 2  Sun  Home      W  2.0   
2    2024-09-01  16:00  Premier League   Matchweek 3  Sun  Away      W  3.0   
3    2024-09-14  15:00  Premier League   Matchweek 4  Sat  Home      L  0.0   
4    2024-09-17  21:00    Champions Lg  League phase  Tue  Away      W  3.0   

                         ... Standard               Expected               \
    GA         Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh   
0  0.0     Ipswich Town  ...     14.8  0.0  0     0      2.6  2.6    0.15   
1  0.0        Brentford  ...     13.6  1.0  0     0      2.5  2.5    0.14   
2  0.0   Manchester Utd  ...     13.4  0.0  0     0      1.8  1.8    0.16   
3  1.0  Nott'ham Forest  ...     14.9  0.0  0     0      0.9  0.9    0.07   
4  1.0         it Milan  ...     15.7  1.0  0     0      3.1  3.1    0.14   

               Unnamed: 25_level_0  
  G-xG np:G-xG        Match Report  
0 -0.6    -0.6        Match Report  
1 -0.5    -0.5        Match Report  
2  1.2     1.2        Match Report  
3 -0.9    -0.9        Match Report  
4 -0.1    -0.1        Match Report  

[5 rows x 26 columns]